In [2]:
import keras
import pandas as pd
import numpy as np
import seaborn as sns

Using TensorFlow backend.


In [3]:
stocks = pd.read_csv('Stock Market/output.csv')

In [4]:
stocks.head()

,Unnamed: 0,rsi14,rsi9,rsi5,macd12,macdhist12,macd6,macdhist6,ema10,ema20,...,arronup7,arrondown13,arronup13,bbindic,close,high,low,open,volume,sym
0,2008-02-06 03:45:00,37.087456,35.387474,34.769877,-11.918355,-0.720816,-6.017640,0.963975,150.694345,160.687525,...,71.428571,15.384615,0.000000,0.281697,143.510803,146.603168,138.623438,142.711058,966300.0,ADANIPORTS.NS
1,2008-02-07 03:45:00,36.233642,34.008069,31.696320,-11.922653,-0.580091,-6.047231,0.560630,149.024724,158.861230,...,57.142857,7.692308,0.000000,0.273414,141.511429,147.509558,140.756103,146.620947,1064660.0,ADANIPORTS.NS
2,2008-02-08 03:45:00,35.284951,32.464190,28.304606,-11.963115,-0.496443,-6.128271,0.287754,147.264444,157.002368,...,42.857143,0.000000,15.384615,0.260618,139.343185,147.509528,135.984235,143.955082,1425125.0,ADANIPORTS.NS
3,2008-02-11 03:45:00,30.200191,24.877477,15.736142,-12.891299,-1.139701,-7.777336,-0.816787,143.470202,154.087487,...,28.571429,100.000000,7.692308,0.102090,126.396111,139.512012,123.517004,139.512012,1119775.0,ADANIPORTS.NS
4,2008-02-12 03:45:00,28.391773,22.454518,12.816413,-13.895499,-1.715121,-9.228652,-1.360862,139.399660,150.944128,...,14.285714,100.000000,0.000000,0.052893,121.082222,130.617035,116.514759,128.848693,1517045.0,ADANIPORTS.NS


In [5]:
from sklearn.model_selection import train_test_split

In [7]:
x = stocks.drop('returns', axis=1).iloc[1:,1:]
y = stocks.returns

In [55]:
trainx = np.array([])
trainy = np.array([])
testx = np.array([])
testy = np.array([])
syms = np.unique(stocks.sym.values)
for symbol in syms:
    data = stocks[stocks.sym == symbol]
    data.loc[:,'returns'] = data.close.pct_change()
    x = data.drop(['sym','Unnamed: 0','returns'], axis=1).values
    y = data.returns.values
    if trainx.shape[0]==0:
        trainx = x[1:2001]
        trainy = y[1:2001]
        testx = x[2001:]
        testy = y[2001:]
    else:
        trainx = np.concatenate((trainx, x[1:2001]), axis=0)
        trainy = np.concatenate((trainy, y[1:2001]), axis=0)
        testx = np.concatenate((testx, x[2001:]), axis=0)
        testy = np.concatenate((testy, y[2001:]), axis=0)

/home/rushim245/anaconda2/envs/DeepLearning/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [59]:
testy.shape

(20611,)

In [29]:
trainx = np.array([])
dat = data.drop(['returns','sym','Unnamed: 0'], axis=1).loc[1:,:]

In [41]:
dat.values.shape

(2396, 22)

In [8]:
trainx, testx, trainy, testy = train_test_split(x,y, test_size=0.2)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
trainx = sc.fit_transform(trainx)
testx = sc.transform(testx)

In [10]:
trainx.shape

(94645, 22)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [19]:
def build_classifier(optimizer):
    # Initializing the layers
    classifier = Sequential()
    
    # Adding input layer and hidden layer
    classifier.add(Dense(512, input_dim=22, kernel_initializer='uniform', activation='relu' ))
    classifier.add(Dropout(0.25))

    # Adding second hidden layer
    classifier.add(Dense(256, kernel_initializer='uniform', activation='relu' ))
    classifier.add(Dropout(0.20))

    # Adding second hidden layer
    classifier.add(Dense(124, kernel_initializer='uniform', activation='relu' ))
    classifier.add(Dropout(0.20))

    # Adding second hidden layer
    classifier.add(Dense(62, kernel_initializer='uniform', activation='relu' ))
    classifier.add(Dropout(0.20))    
    
    # Adding output layer
    classifier.add(Dense(1, kernel_initializer='uniform', activation='sigmoid' ))
    
    # ANN Compile
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return classifier

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

classifier =  build_classifier('adam')

In [21]:
classifier.fit(trainx, trainy, epochs=600, batch_size=60, verbose=0)

In [26]:
predy = classifier.predict(testx)

In [34]:
pd.DataFrame({'testy':testy.values, 'predict':predy.values})

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [27]:
predy = (predy > 0.5)

In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
confusion_matrix(predy, testy)

array([[21456,  1848],
       [  260,    98]])